# Splitting dataset into train, test and valid subsets 

In [8]:
import os
import numpy as np
import pandas as pd
import shutil
from tqdm import tqdm

In [9]:
def split_dataset_and_files(
    metadata_path: str,
    audio_folder: str,
    output_folder: str,
    train_ratio: float = 0.7,
    val_ratio: float   = 0.15,
    test_ratio: float  = 0.15
):
    """
    Dzieli zbiór chunków (audio_30_s_chunks_metadata) na train/val/test
    w proporcjach 70-15-15 (domyślnie), tak aby pary (ARTIST_ID, ALBUM_ID)
    trafiały do jednego zbioru.
    
    Następnie tworzy:
      - train_metadata.csv, val_metadata.csv, test_metadata.csv
        (bez zmiany PATH - wciąż np. "76/1371076_chunk_0.mp3")
      - kopiuje pliki MP3 do:
          output_folder/train/...,
          output_folder/val/...,
          output_folder/test/...,
        zachowując oryginalną strukturę podfolderów (np. "76/").
    """

    # 1. Wczytujemy metadane
    metadata = pd.read_csv(metadata_path)

    # Sprawdzamy czy sumy proporcji = 1.0
    if not np.isclose(train_ratio + val_ratio + test_ratio, 1.0):
        raise ValueError("Proporcje zbiorów (train_ratio, val_ratio, test_ratio) muszą sumować się do 1.0.")

    # Upewniamy się, że istnieją kolumny: ARTIST_ID, ALBUM_ID, PATH
    required_cols = ["ARTIST_ID", "ALBUM_ID", "PATH"]
    for col in required_cols:
        if col not in metadata.columns:
            raise ValueError(f"Brak wymaganej kolumny '{col}' w pliku {metadata_path}.")

    # 2. Grupujemy po (ARTIST_ID, ALBUM_ID)
    group_keys = list(metadata.groupby(["ARTIST_ID", "ALBUM_ID"]).groups.keys())  # lista par (artist, album)

    # 3. Tasujemy i dzielimy listę grup w stosunku 70-15-15
    np.random.shuffle(group_keys)
    n = len(group_keys)
    train_end = int(n * train_ratio)
    val_end   = train_end + int(n * val_ratio)

    train_keys = set(group_keys[:train_end])
    val_keys   = set(group_keys[train_end:val_end])
    test_keys  = set(group_keys[val_end:])

    # 4. Przypisujemy subset (train/val/test) na podstawie par (ARTIST_ID, ALBUM_ID)
    def assign_subset(row):
        key = (row["ARTIST_ID"], row["ALBUM_ID"])
        if key in train_keys:
            return "train"
        elif key in val_keys:
            return "val"
        elif key in test_keys:
            return "test"
        else:
            return "unknown"

    metadata["subset"] = metadata.apply(assign_subset, axis=1)

    # Dzielimy na 3 DataFrame
    train_data = metadata[metadata["subset"] == "train"].copy()
    val_data   = metadata[metadata["subset"] == "val"].copy()
    test_data  = metadata[metadata["subset"] == "test"].copy()

    # Sprawdzamy czy sumy się zgadzają
    assert len(train_data) + len(val_data) + len(test_data) == len(metadata), \
        "Suma (train + val + test) nie zgadza się z całkowitą liczbą chunków!"

    # 5. Tworzymy foldery docelowe (train, val, test)
    os.makedirs(os.path.join(output_folder, "train"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "val"),   exist_ok=True)
    os.makedirs(os.path.join(output_folder, "test"),  exist_ok=True)

    # 6. Funkcja do kopiowania plików z tqdm
    def copy_files(df_subset, subset_name):
        """
        Kopiuje pliki z audio_folder + PATH -> output_folder/subset_name + PATH
        zachowując strukturę (np. "76/1371076_chunk_0.mp3").
        Nie modyfikuje kolumny PATH w df_subset.
        """
        subset_folder = os.path.join(output_folder, subset_name)
        
        # Pasek postępu
        for _, row_ in tqdm(df_subset.iterrows(),
                            total=len(df_subset),
                            desc=f"Kopiowanie do {subset_name}",
                            unit="plik"):
            rel_path = row_["PATH"]  # np. "76/1371076_chunk_0.mp3"
            src  = os.path.join(audio_folder, rel_path)
            dest = os.path.join(subset_folder, rel_path)

            os.makedirs(os.path.dirname(dest), exist_ok=True)

            if os.path.exists(src):
                shutil.copy2(src, dest)
            else:
                print(f"[WARNING] Brak pliku: {src}")

    print("\nRozpoczynam kopiowanie plików do train/val/test...")

    # Kopiujemy pliki do poszczególnych folderów
    copy_files(train_data, "train")
    copy_files(val_data,   "val")
    copy_files(test_data,  "test")

    # 7. Zwracamy lub zapisujemy zbiory w postaci CSV
    return train_data, val_data, test_data

In [10]:
metadata_path = "../../datasets/jamendo/metadata/audio_30_s_chunks_metadata.csv"
audio_folder  = "../../datasets/jamendo/audio_30_s_chunks/"
output_folder = "../../datasets/jamendo/split_audio_dataset/"

# Wywołujemy podział
train_data, val_data, test_data = split_dataset_and_files(
    metadata_path,
    audio_folder,
    output_folder,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

# Zapis do plików CSV
train_data.to_csv("../../datasets/jamendo/metadata/train_metadata.csv", index=False)
val_data.to_csv("../../datasets/jamendo/metadata/val_metadata.csv",   index=False)
test_data.to_csv("../../datasets/jamendo/metadata/test_metadata.csv", index=False)

print("\nZbiory danych i pliki audio zostały podzielone i zapisane.")
print(f"- Zbiór treningowy: {len(train_data)} przykładów")
print(f"- Zbiór walidacyjny: {len(val_data)} przykładów")
print(f"- Zbiór testowy:    {len(test_data)} przykładów")


Rozpoczynam kopiowanie plików do train/val/test...


Kopiowanie do test: 100%|██████████| 6372/6372 [00:08<00:00, 715.29plik/s]



Zbiory danych i pliki audio zostały podzielone i zapisane.
- Zbiór treningowy: 26743 przykładów
- Zbiór walidacyjny: 5982 przykładów
- Zbiór testowy:    6372 przykładów
